<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/colab/1-SQL/G2-Inserting_Data_in_MySQL_using_PythonPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [4]:
len(data)

1289

In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head(1)


,electric_bike_surcharge_waiver,eightd_station_services,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,rental_methods,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,False,[],classic,40.767272,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...","[CREDITCARD, KEY]",False,True,72,6926.01,72


In [6]:
# We drop the 'rental methods' column, as it contains multiple values and 
# we cannot insert lists in a database cell.
# also drop eightd_station_services for the same reason
df.drop('rental_methods', axis=1, inplace = True)
df.drop('eightd_station_services', axis=1, inplace = True)
df.head(5)

,electric_bike_surcharge_waiver,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,False,classic,40.767272,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,72,6926.01,72
1,False,classic,40.719116,71,Franklin St & W Broadway,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.006667,33,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,79,5430.08,79
2,False,classic,40.711174,71,St James Pl & Pearl St,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.000165,27,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,82,5167.06,82
3,False,classic,40.683826,71,Atlantic Ave & Fort Greene Pl,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.976323,62,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,83,4354.07,83
4,False,classic,40.741776,71,W 17 St & 8 Ave,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.001497,50,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,116,6148.02,116


### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [7]:
!sudo -H pip3 install -U sqlalchemy

Requirement already up-to-date: sqlalchemy in /usr/local/lib/python3.6/dist-packages (1.3.22)


In [9]:
!sudo apt-get install -y python-dev libmysqlclient-dev && sudo pip install mysqlclient

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following NEW packages will be installed:
  libmysqlclient-dev
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 988 kB of archives.
After this operation, 6,013 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient-dev amd64 5.7.32-0ubuntu0.18.04.1 [988 kB]
Fetched 988 kB in 1s (843 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting 

In [29]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'bigdata.stern.nyu.edu', 
    user = 'DealingS21',
    password = 'DealingS21!!')

engine = create_engine(conn_string)
con = engine.connect()

Once we have connected successfully, we need to create our database:

In [30]:
# Query to create a database
db_name = 'citibike_stations'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
engine.execute(create_db_query)
# And lets switch to the database
engine.execute("USE {db}".format(db=db_name))

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [36]:
# Drop the table Stations, just to be sure
drop_table_query= "drop  table Stations "
engine.execute(drop_table_query)

In [37]:
# This step is typically optional, but it is good practice to define explicitly the 
# data types before storing things in a database. In many cases, this can be ommitted, though.
import sqlalchemy
dtype = {
    'capacity': sqlalchemy.types.INT,
    'eightd_has_key_dispenser':  sqlalchemy.types.BOOLEAN,
    'lat': sqlalchemy.types.Float, 
    'lon': sqlalchemy.types.Float,
    'name': sqlalchemy.types.VARCHAR(100),
    'region_id': sqlalchemy.types.VARCHAR(5),
    'rental_url': sqlalchemy.types.VARCHAR(100),
    'short_name': sqlalchemy.types.VARCHAR(10),
    'station_id': sqlalchemy.types.INT
}

#### Let's look at our dataframe

In [38]:
df.head(3)

,electric_bike_surcharge_waiver,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,False,classic,40.767272,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,72,6926.01,72
1,False,classic,40.719116,71,Franklin St & W Broadway,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.006667,33,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,79,5430.08,79
2,False,classic,40.711174,71,St James Pl & Pearl St,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.000165,27,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,82,5167.06,82


#### The to_sql method gives an error, but actually creates the table

In [39]:
table_name = 'Stations'



df.head(5)

# Create a table (this gives an error, but actually creates the Stations table)
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation
df.to_sql(table_name, engine,  index = False,  dtype=dtype)
    


AttributeError: ignored

In [40]:
df.head(5)

,electric_bike_surcharge_waiver,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,False,classic,40.767272,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,72,6926.01,72
1,False,classic,40.719116,71,Franklin St & W Broadway,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.006667,33,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,79,5430.08,79
2,False,classic,40.711174,71,St James Pl & Pearl St,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.000165,27,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,82,5167.06,82
3,False,classic,40.683826,71,Atlantic Ave & Fort Greene Pl,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.976323,62,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,83,4354.07,83
4,False,classic,40.741776,71,W 17 St & 8 Ave,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.001497,50,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",False,True,116,6148.02,116


In [41]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = 'ALTER TABLE Stations ADD PRIMARY KEY(station_id)'
engine.execute(add_key_query)

### Reading from a SQL Database in Python using Pandas

We can similarly read from the database using Pandas

In [42]:
query =  "SELECT * FROM Stations"
df2 = pd.read_sql(query, con=engine)
df2.head(5)


,electric_bike_surcharge_waiver,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,0,classic,40.7673,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.9939,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0,1,72,6926.01,72
1,0,classic,40.7191,71,Franklin St & W Broadway,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.0067,33,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0,1,79,5430.08,79
2,0,classic,40.7112,71,St James Pl & Pearl St,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.0002,27,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0,1,82,5167.06,82
3,0,classic,40.6838,71,Atlantic Ave & Fort Greene Pl,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.9763,62,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0,1,83,4354.07,83
4,0,classic,40.7418,71,W 17 St & 8 Ave,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.0015,50,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0,1,116,6148.02,116


And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [43]:
# The necessary library to write in Excel
!pip3 install -U xlwt

Requirement already up-to-date: xlwt in /usr/local/lib/python3.6/dist-packages (1.3.0)


### Save the table in CSV and xls format 


In [45]:
df2.to_excel("citibike.xls")
df2.to_csv("citibike.csv")
!ls
!head citibike.csv

citibike.csv  citibike.xls  sample_data
,electric_bike_surcharge_waiver,station_type,lat,region_id,name,external_id,lon,capacity,rental_uris,eightd_has_key_dispenser,has_kiosk,station_id,short_name,legacy_id
0,0,classic,40.7673,71,W 52 St & 11 Ave,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.9939,55,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan', 'android': 'https://bkn.lft.to/lastmile_qr_scan'}",0,1,72,6926.01,72
1,0,classic,40.7191,71,Franklin St & W Broadway,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.0067,33,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan', 'android': 'https://bkn.lft.to/lastmile_qr_scan'}",0,1,79,5430.08,79
2,0,classic,40.7112,71,St James Pl & Pearl St,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.0002,27,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan', 'android': 'https://bkn.lft.to/lastmile_qr_scan'}",0,1,82,5167.06,82
3,0,classic,40.6838,71,Atlantic Ave & Fort Greene Pl,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.9763,62,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan', 'android':

### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above. You should use the database for your team.